In [16]:
from loading.loadIDS import loadIDS, trainIDS
from loading.loadGAN import load_gan, load_gan_kdd
from loading.loadingKDD import loadingKDD
from generation.generation import generation_fake_data
from training.training_gan import train_gan, train_gan_kdd
from matplotlib import pyplot as plt
import numpy as np

# Parameters

# DATA
# x_train, y_train, x_test, y_test = loadData(nrows=100000, attacks=True)
attack_mode = None
X, Y, cat_col, colnames = loadingKDD(nrows=100000000, attack_mode=attack_mode, attack=None)


/home/peseux/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [17]:
features = ["num_access_files", "num_shells", "src_bytes", "dst_bytes", "root_shell", "num_root", "su_attempted", "num_file_creations"]

In [18]:
def creating_dico_index(features, colnames):
    output = dict()
    for col in colnames:
        output[col] = [i for i in range(122) if colnames[i] == col][0]
    return output
dico_index = creating_dico_index(features=features, colnames=colnames)

In [32]:
def CVSS(naf, ns, sb, db, rs, nr,sa,nfc):
    access_vector = naf
    attack_complexity = (ns+2)*(sb+db)/2
    authentification = (rs+nr)*((2+sa)++2)
    conf_impact = naf
    int_impact = nfc
    availibility_impact = (sb+db)/2
    exploitability = 20*access_vector *attack_complexity*authentification
    impact = 10.41 * (1-(1-conf_impact)*(1-int_impact)*(1-availibility_impact))
    f_impact = 1.17 * (impact > 0)
    
    return (0.6*impact + 0.4*exploitability -1.5)*f_impact

def score_line(x, dico_index):
    values = dict()
    for feature in dico_index:
        values[feature] = x[dico_index[feature]]
    score = CVSS(naf=values["num_access_files"], 
                 ns=values["num_shells"], 
                 sb=values["src_bytes"], 
                 db=values["dst_bytes"], 
                 rs=values["root_shell"], 
                 nr=values["num_root"],
                 sa=values["su_attempted"],
                 nfc=values["num_file_creations"])
    return score

In [33]:
X[0].shape

(122,)

In [34]:
score_line(x=X[0], dico_index=dico_index)

-83.91944758494974